In [1]:
import wget
import newspaper
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
from pandarallel import pandarallel
pandarallel.initialize()

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
sitemap = 'https://www.nbcnews.com/sitemap/nbcnews/sitemap-index'
coverpage = requests.get(sitemap).content
data = xmltodict.parse(coverpage)
pages = data['sitemapindex']['sitemap']
results = [i['loc'] for i in pages]
df = pd.DataFrame(results, columns = ['page'])
df = df[df.page.str.contains('article')].reset_index(drop=True)
df

,page
0,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml
1,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-03-article.xml
2,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-02-article.xml
3,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-01-article.xml
4,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2020-12-article.xml
...,...
215,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-05-article.xml
216,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-04-article.xml
217,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-03-article.xml
218,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-02-article.xml


In [24]:
headers={
    'accept': '*/*',
#     'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

def get_articles(page):
    coverpage = requests.get(page,headers = headers).content
    time.sleep(2)
    data = xmltodict.parse(coverpage)
    articles = data['urlset']['url']
    return [str(i) for i in articles]

In [25]:
pandarallel.initialize(nb_workers= 2, progress_bar=True)
df['articles']= None
df['articles']= df['page'].parallel_apply(get_articles)

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [26]:
df.to_csv('nbc_1.csv', index=False)

In [4]:
df= pd.read_csv('nbc_1.csv', header=0)
df

,page,articles
0,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"[""OrderedDict([('loc', 'https://www.nbcnews.com/think/opinion/new-york-mayoral-race-polls-put-andrew-yang-ahead-his-ncna1264558'), ('lastmod', '2021-04-20T08:30:40.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-04-20T08:30:39Z'), ('news:title', 'New York mayoral race polls put Andrew Yang ahead. His policies would put NYC behind.')])), ('image:image', OrderedDict([('image:loc', 'https://media2.s-nbcnews.com/i/newscms/2021_16/3465995/210419-think-andrew-yang-bad-for-ny-se-500p_cab..."
1,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-03-article.xml,"[""OrderedDict([('loc', 'https://www.nbcnews.com/news/us-news/these-states-are-rolling-back-covid-restrictions-including-mask-mandates-n1259751'), ('lastmod', '2021-04-19T20:28:37Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-03-31T19:31:23Z'), ('news:title', 'These states are rolling back Covid restrictions, including mask mandates and indoor capacity caps')])), ('image:image', OrderedDict([('image:loc', 'https://media1.s-nbcnews.com/i/newscms/2021_09/3454703/210305-texas-covid-mask..."
2,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-02-article.xml,"[""OrderedDict([('loc', 'https://www.nbcnews.com/health/health-news/j-j-covid-19-vaccine-distribution-begin-immediately-n1259101'), ('lastmod', '2021-03-01T00:49:43.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-02-28T23:55:00Z'), ('news:title', 'J&J Covid-19 vaccine distribution to begin immediately')])), ('image:image', OrderedDict([('image:loc', 'https://media2.s-nbcnews.com/i/newscms/2021_08/3453326/210228-johnson-johnson-vaccine-jm-1944_a96ca3fb345b5a13d118d5c3391c0b04.jpg')]..."
3,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-01-article.xml,"['OrderedDict([(\'loc\', \'https://www.nbcnews.com/politics/congress/parkland-victim-s-mother-says-she-spoke-marjorie-taylor-greene-n1256294\'), (\'lastmod\', \'2021-02-01T16:35:32.000Z\'), (\'news:news\', OrderedDict([(\'news:publication\', OrderedDict([(\'news:name\', \'NBCNews.com\'), (\'news:language\', \'en\')])), (\'news:publication_date\', \'2021-01-31T21:49:00Z\'), (\'news:title\', ""Parkland victim\'s mother says she spoke with Marjorie Taylor Greene about school shooting conspiracies"")]))])', 'OrderedDict([(\'loc\', \'https://www.nbcnews.com/news/weather/major-nor-easter-expected-..."
4,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2020-12-article.xml,"[""OrderedDict([('loc', 'https://www.nbcnews.com/know-your-value/feature/stylist-monica-barnett-how-create-awe-inspiring-closet-career-success-ncna1252626'), ('lastmod', '2020-12-31T23:59:51.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2020-12-31T23:59:51Z'), ('news:title', 'Stylist Monica Barnett: How to create an awe-inspiring closet for career success in the new year')])), ('image:image', OrderedDict([('image:loc', 'https://media3.s-nbcnews.com/i/newscms/2020_53/3438992/closet_843..."
...,...,...
215,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-05-article.xml,"[""OrderedDict([('loc', 'https://www.nbcnews.com/id/wbna3078430'), ('lastmod', '2003-05-30T19:44:20.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2003-05-30T19:44:20Z'), ('news:title', 'Video games could be good for you')]))])"", ""OrderedDict([('loc', 'https://www.nbcnews.com/id/wbna3078930'), ('lastmod', '2003-05-30T13:16:44.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com')

In [5]:
df.articles = df.articles.parallel_apply(eval)
df = df.explode('articles').reset_index(drop=True)
df

,page,articles
0,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/think/opinion/new-york-mayoral-race-polls-put-andrew-yang-ahead-his-ncna1264558'), ('lastmod', '2021-04-20T08:30:40.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-04-20T08:30:39Z'), ('news:title', 'New York mayoral race polls put Andrew Yang ahead. His policies would put NYC behind.')])), ('image:image', OrderedDict([('image:loc', 'https://media2.s-nbcnews.com/i/newscms/2021_16/3465995/210419-think-andrew-yang-bad-for-ny-se-500p_cabd3..."
1,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/news/us-news/derek-chauvin-trial-minneapolis-nation-anxiously-await-verdict-n1264582'), ('lastmod', '2021-04-20T08:30:10.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-04-20T08:30:10Z'), ('news:title', 'Derek Chauvin trial: Minneapolis and a nation anxiously await verdict')]))])"
2,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/politics/justice-department/we-found-hundreds-posts-about-plans-attack-capitol-why-aren-n1264291'), ('lastmod', '2021-04-20T08:31:09.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-04-20T08:30:00Z'), ('news:title', ""We found hundreds of posts about plans to attack the Capitol. Why aren't we seeing this evidence in court?"")])), ('image:image', OrderedDict([('image:loc', 'https://media4.s-nbcnews.com/i/newscms/2021_05/3446596/210201-prou..."
3,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/news/us-news/montana-backcountry-guide-dies-after-he-was-mauled-grizzly-near-n1264590'), ('lastmod', '2021-04-20T05:54:41.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-04-20T05:54:40Z'), ('news:title', 'Montana backcountry guide dies after he was mauled by grizzly near Yellowstone')])), ('image:image', OrderedDict([('image:loc', 'https://media3.s-nbcnews.com/i/newscms/2021_16/3466093/bear_mauling_54216-jpg-7b5ce_d3f300937606bccde0aec..."
4,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/news/us-news/14-people-rescued-after-boat-sinks-southern-california-coast-n1264589'), ('lastmod', '2021-04-20T04:57:19.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2021-04-20T04:57:18Z'), ('news:title', '14 people rescued after boat sinks off Southern California coast')]))])"
...,...,...
662471,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-01-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/id/wbna3071782'), ('lastmod', '2004-02-27T22:17:10.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2003-01-06T06:34:33Z'), ('news:title', 'Gulf War lessons, learned or not')])), ('image:image', OrderedDict([('image:loc', 'http://media2.s-nbcnews.com/i/msnbc/1744000/1744799.jpg')]))])"
662472,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-01-article.xml,"OrderedDict([('loc', 'https://www.nbcnews.com/id/wbna3078286'), ('lastmod', '2003-01-03T16:52:36.000Z'), ('news:news', OrderedDict([('news:publication', OrderedDict([('news:name', 'NBCNews.com'), ('news:language', 'en')])), ('news:publication_date', '2003-01-03T16:52:36Z'), ('news:title', 'Two super cell phones worth waiting for')]))])"
662473,https:

In [6]:
import collections
from collections import OrderedDict
import ast

# def convert(line):
#     values = re.search(r"OrderedDict\((.*)\)", line).group(1)
#     return dict(eval(values))

In [7]:
df['articles'] = df['articles'].parallel_apply(lambda x: re.search(r"OrderedDict\((.*)\)", x).group(1))
df['articles'] = df['articles'].parallel_apply(lambda x: OrderedDict(eval(x)))
df

,page,articles
0,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"{'loc': 'https://www.nbcnews.com/think/opinion/new-york-mayoral-race-polls-put-andrew-yang-ahead-his-ncna1264558', 'lastmod': '2021-04-20T08:30:40.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2021-04-20T08:30:39Z', 'news:title': 'New York mayoral race polls put Andrew Yang ahead. His policies would put NYC behind.'}, 'image:image': {'image:loc': 'https://media2.s-nbcnews.com/i/newscms/2021_16/3465995/210419-think-andrew-yang-bad-for-ny-se-500p_cabd3f206bcd1483e8a95d4c68144cdd.jpg'}}"
1,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"{'loc': 'https://www.nbcnews.com/news/us-news/derek-chauvin-trial-minneapolis-nation-anxiously-await-verdict-n1264582', 'lastmod': '2021-04-20T08:30:10.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2021-04-20T08:30:10Z', 'news:title': 'Derek Chauvin trial: Minneapolis and a nation anxiously await verdict'}}"
2,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"{'loc': 'https://www.nbcnews.com/politics/justice-department/we-found-hundreds-posts-about-plans-attack-capitol-why-aren-n1264291', 'lastmod': '2021-04-20T08:31:09.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2021-04-20T08:30:00Z', 'news:title': 'We found hundreds of posts about plans to attack the Capitol. Why aren't we seeing this evidence in court?'}, 'image:image': {'image:loc': 'https://media4.s-nbcnews.com/i/newscms/2021_05/3446596/210201-proud-boys-capitol-ew-1127a_f3159ad2a01afd6f193a251280531003.jpg'}}"
3,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"{'loc': 'https://www.nbcnews.com/news/us-news/montana-backcountry-guide-dies-after-he-was-mauled-grizzly-near-n1264590', 'lastmod': '2021-04-20T05:54:41.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2021-04-20T05:54:40Z', 'news:title': 'Montana backcountry guide dies after he was mauled by grizzly near Yellowstone'}, 'image:image': {'image:loc': 'https://media3.s-nbcnews.com/i/newscms/2021_16/3466093/bear_mauling_54216-jpg-7b5ce_d3f300937606bccde0aec6aed489cac2.jpg'}}"
4,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2021-04-article.xml,"{'loc': 'https://www.nbcnews.com/news/us-news/14-people-rescued-after-boat-sinks-southern-california-coast-n1264589', 'lastmod': '2021-04-20T04:57:19.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2021-04-20T04:57:18Z', 'news:title': '14 people rescued after boat sinks off Southern California coast'}}"
...,...,...
662471,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-01-article.xml,"{'loc': 'https://www.nbcnews.com/id/wbna3071782', 'lastmod': '2004-02-27T22:17:10.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2003-01-06T06:34:33Z', 'news:title': 'Gulf War lessons, learned or not'}, 'image:image': {'image:loc': 'http://media2.s-nbcnews.com/i/msnbc/1744000/1744799.jpg'}}"
662472,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-01-article.xml,"{'loc': 'https://www.nbcnews.com/id/wbna3078286', 'lastmod': '2003-01-03T16:52:36.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2003-01-03T16:52:36Z', 'news:title': 'Two super cell phones worth waiting for'}}"
662473,https://www.nbcnews.com/sitemap/nbcnews/sitemap-2003-01-article.xml,"{'loc': 'https://www.nbcnews.com/id/wbna3073329', 'lastmod': '2003-10-31T22:33:09.000Z', 'news:news': {'news:publication': {'news:name': 'NBCNews.com', 'news:language': 'en'}, 'news:publication_date': '2003-01-03T14:29:02Z', 'news:title': 'Another hot year for automakers'},

In [8]:
df = pd.json_normalize(df.articles)
df.columns= ['url','lastmod','name','language','date','title','image']
df = df[['url','date','title']]
df

,url,date,title
0,https://www.nbcnews.com/think/opinion/new-york-mayoral-race-polls-put-andrew-yang-ahead-his-ncna1264558,2021-04-20T08:30:39Z,New York mayoral race polls put Andrew Yang ahead. His policies would put NYC behind.
1,https://www.nbcnews.com/news/us-news/derek-chauvin-trial-minneapolis-nation-anxiously-await-verdict-n1264582,2021-04-20T08:30:10Z,Derek Chauvin trial: Minneapolis and a nation anxiously await verdict
2,https://www.nbcnews.com/politics/justice-department/we-found-hundreds-posts-about-plans-attack-capitol-why-aren-n1264291,2021-04-20T08:30:00Z,We found hundreds of posts about plans to attack the Capitol. Why aren't we seeing this evidence in court?
3,https://www.nbcnews.com/news/us-news/montana-backcountry-guide-dies-after-he-was-mauled-grizzly-near-n1264590,2021-04-20T05:54:40Z,Montana backcountry guide dies after he was mauled by grizzly near Yellowstone
4,https://www.nbcnews.com/news/us-news/14-people-rescued-after-boat-sinks-southern-california-coast-n1264589,2021-04-20T04:57:18Z,14 people rescued after boat sinks off Southern California coast
...,...,...,...
662471,https://www.nbcnews.com/id/wbna3071782,2003-01-06T06:34:33Z,"Gulf War lessons, learned or not"
662472,https://www.nbcnews.com/id/wbna3078286,2003-01-03T16:52:36Z,Two super cell phones worth waiting for
662473,https://www.nbcnews.com/id/wbna3073329,2003-01-03T14:29:02Z,Another hot year for automakers
662474,https://www.nbcnews.com/id/wbna3073316,2003-01-03T12:30:29Z,Wall Street’s premature celebration


In [9]:
df.to_csv('nbc_2.csv', index=False)

In [11]:
df.date = pd.to_datetime(df.date)

In [14]:
df = df[df.date<'2013-04-01'].reset_index(drop=True)
df

,url,date,title
0,https://www.nbcnews.com/news/photo/louisville-player-after-horrific-injury-dont-worry-about-me-ill-flna1C9155862,2013-03-31 23:11:05+00:00,Louisville player after horrific injury: 'Don't worry about me. I'll be OK. You guys go win this thing'
1,https://www.nbcnews.com/news/photo/louisville-player-after-horrific-injury-dont-worry-about-me-ill-flna1C9156650,2013-03-31 23:11:05+00:00,Louisville player after horrific injury: 'Don't worry about me. I'll be OK. You guys go win this thing'
2,https://usnews.nbcnews.com/_news/2013/03/31/17542064-philadelphia-father-of-two-shoots-kills-home-intruder,2013-03-31 21:58:35+00:00,"Philadelphia father of two shoots, kills home intruder"
3,https://www.nbcnews.com/business/business-news/crude-oil-leaks-arkansas-suburb-after-exxonmobil-pipeline-ruptures-flna1C9155011,2013-03-31 20:51:00+00:00,Crude oil leaks in Arkansas suburb after ExxonMobil pipeline ruptures
4,https://www.nbcnews.com/news/us-news/dust-chinese-storm-reaches-central-california-flna1C9155498,2013-03-31 20:47:51+00:00,Dust from Chinese storm reaches central California
...,...,...,...
418231,https://www.nbcnews.com/id/wbna3071782,2003-01-06 06:34:33+00:00,"Gulf War lessons, learned or not"
418232,https://www.nbcnews.com/id/wbna3078286,2003-01-03 16:52:36+00:00,Two super cell phones worth waiting for
418233,https://www.nbcnews.com/id/wbna3073329,2003-01-03 14:29:02+00:00,Another hot year for automakers
418234,https://www.nbcnews.com/id/wbna3073316,2003-01-03 12:30:29+00:00,Wall Street’s premature celebration


In [15]:
df.to_csv('nbc_2003_2013.csv', index=False)

In [19]:
df = pd.read_csv('nbc_2003_2013.csv', header=0)

#  filtering

In [20]:
with open('wiki_relevant.txt', 'r') as file:    
    relevant_rules = file.read().split('\n')
relevant_rules = '\\b' + '|\\b'.join(relevant_rules)

if relevant_rules.endswith('|\\b'):
    relevant_rules =relevant_rules[:-3]   

relevant_rules = re.compile(relevant_rules, re.IGNORECASE)


with open('irelevant_keywords.txt', 'r') as file:    
    irelevant_rules = file.read().split('\n')
irelevant_rules = '\\b' + '|\\b'.join(irelevant_rules)

if irelevant_rules.endswith('|\\b'):
    irelevant_rules =irelevant_rules[:-3]   

irelevant_rules = re.compile(irelevant_rules, re.IGNORECASE)

find_relevant = lambda x: dict(Counter(relevant_rules.findall(str(x).lower())))
find_exclude = lambda x: dict(Counter(irelevant_rules.findall(str(x).lower())))

In [21]:
df

,url,date,title
0,https://www.nbcnews.com/news/photo/louisville-player-after-horrific-injury-dont-worry-about-me-ill-flna1C9155862,2013-03-31 23:11:05+00:00,Louisville player after horrific injury: 'Don't worry about me. I'll be OK. You guys go win this thing'
1,https://www.nbcnews.com/news/photo/louisville-player-after-horrific-injury-dont-worry-about-me-ill-flna1C9156650,2013-03-31 23:11:05+00:00,Louisville player after horrific injury: 'Don't worry about me. I'll be OK. You guys go win this thing'
2,https://usnews.nbcnews.com/_news/2013/03/31/17542064-philadelphia-father-of-two-shoots-kills-home-intruder,2013-03-31 21:58:35+00:00,"Philadelphia father of two shoots, kills home intruder"
3,https://www.nbcnews.com/business/business-news/crude-oil-leaks-arkansas-suburb-after-exxonmobil-pipeline-ruptures-flna1C9155011,2013-03-31 20:51:00+00:00,Crude oil leaks in Arkansas suburb after ExxonMobil pipeline ruptures
4,https://www.nbcnews.com/news/us-news/dust-chinese-storm-reaches-central-california-flna1C9155498,2013-03-31 20:47:51+00:00,Dust from Chinese storm reaches central California
...,...,...,...
418231,https://www.nbcnews.com/id/wbna3071782,2003-01-06 06:34:33+00:00,"Gulf War lessons, learned or not"
418232,https://www.nbcnews.com/id/wbna3078286,2003-01-03 16:52:36+00:00,Two super cell phones worth waiting for
418233,https://www.nbcnews.com/id/wbna3073329,2003-01-03 14:29:02+00:00,Another hot year for automakers
418234,https://www.nbcnews.com/id/wbna3073316,2003-01-03 12:30:29+00:00,Wall Street’s premature celebration


In [22]:
from collections import Counter

head = df['url'] + ' ' + df['title']

df['head_in'] = head.parallel_apply(find_relevant)
df['head_out'] = head.parallel_apply(find_exclude)

head_in = df['head_in'].str.len()
head_out = df['head_out'].str.len()

remove_index =\
    ((head_in ==0) & ( head_out > 0)) |\
    ((head_in > 0) & ( head_out > 1))

remove_index = df[remove_index== True].index
remove_index.shape

(86112,)

In [37]:
# df.date = pd.to_datetime(df.date)
df['date'] = df['date'].dt.strftime('%Y/%m/%d')

/home/yibo/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [40]:
df = df[~df.duplicated(subset='url')]
df = df[~df.duplicated(subset=['title', 'date'])]
df

,url,date,title,head_in,head_out
0,https://www.nbcnews.com/news/photo/louisville-player-after-horrific-injury-dont-worry-about-me-ill-flna1C9155862,2013/03/31,Louisville player after horrific injury: 'Don't worry about me. I'll be OK. You guys go win this thing',{'injur': 2},{'player': 2}
2,https://usnews.nbcnews.com/_news/2013/03/31/17542064-philadelphia-father-of-two-shoots-kills-home-intruder,2013/03/31,"Philadelphia father of two shoots, kills home intruder","{'shoots': 2, 'kill': 2}",{}
3,https://www.nbcnews.com/business/business-news/crude-oil-leaks-arkansas-suburb-after-exxonmobil-pipeline-ruptures-flna1C9155011,2013/03/31,Crude oil leaks in Arkansas suburb after ExxonMobil pipeline ruptures,{},{'business': 2}
4,https://www.nbcnews.com/news/us-news/dust-chinese-storm-reaches-central-california-flna1C9155498,2013/03/31,Dust from Chinese storm reaches central California,{},{}
7,https://www.nbcnews.com/news/us-news/three-dead-95-car-pileup-near-virginia-north-carolina-state-flna1C9155443,2013/03/31,"Three dead in 95-car pileup near Virginia, North Carolina state line",{'dead': 2},{}
...,...,...,...,...,...
418231,https://www.nbcnews.com/id/wbna3071782,2003/01/06,"Gulf War lessons, learned or not",{'war': 1},{}
418232,https://www.nbcnews.com/id/wbna3078286,2003/01/03,Two super cell phones worth waiting for,{},{}
418233,https://www.nbcnews.com/id/wbna3073329,2003/01/03,Another hot year for automakers,{},{}
418234,https://www.nbcnews.com/id/wbna3073316,2003/01/03,Wall Street’s premature celebration,{},{}


In [41]:
num_chunks = 50
df_save = np.array_split(df, num_chunks)
save_chunks = lambda x: df_save[x].to_csv('nbc/%03d.csv'%x, index=False)
pandarallel.initialize(progress_bar=True)       
_ = pd.Series(range(num_chunks)).parallel_apply(save_chunks) 

INFO: Pandarallel will run on 40 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [42]:
del df_save

In [3]:
headers={
    'accept': '*/*',
    'referer': 'https://www.latimes.com/sitemap/',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}
   
from newspaper import Article, Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 10

def extract_story(url):
    try:
        article = Article(url,config= config)
        article.download()
        time.sleep(2)
        article.parse()
        return article.text
    except:
        print('error %s'%url)
        return

# crawling

In [4]:
pandarallel.initialize(nb_workers=4, progress_bar=False)
batch_size= 4*5

for idx in [32,33,34]:
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('nbc/%03d.csv'%idx, header=0)
    
    i=0
    while i < df.shape[0]:
        print(i, end=',')
        df.loc[i: i+ batch_size-1,'text']=\
        df.loc[i: i+ batch_size-1, 'url'].parallel_apply(extract_story).values
        i += batch_size
        if i % 1024:
            df.to_csv('nbc/%03d.csv'%idx, index=False)
    df.to_csv('nbc/%03d.csv'%idx, index=False)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

 ----------32 ------------
0,20,error https://clicked.nbcnews.com/_news/2008/04/15/4355249-when-cameras-lie
40,error https://clicked.nbcnews.com/_news/2008/04/15/4355525-the-return-of-voices-in-my-head-lust-in-translation
60,error https://yourbiz.nbcnews.com/_news/2008/04/15/4355641-yahoo-merger-holds-peril-for-small-biz
error https://clicked.nbcnews.com/_news/2008/04/15/4355248-that-anti-oprah-video
80,100,120,140,160,error https://clicked.nbcnews.com/_news/2008/04/14/4354977-that-harry-potter-fan-site-at-the-root-of-jk-rowlings-lawsuit
180,200,220,240,260,280,300,320,340,360,error https://worldblog.nbcnews.com/_news/2008/04/11/4376290-experts-battle-terror-with-minds-not-guns
error https://clicked.nbcnews.com/_news/2008/04/11/4355340-sometimes-a-link-is-just-a-link
380,error https://clicked.nbcnews.com/_news/2008/04/11/4355430-that-kobe-bryant

In [21]:
for idx in range(50):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('nbc/%03d.csv'%idx, header=0)
    df = df[df.text.str.len()>150]
    df = df[~df.text.duplicated()]
    df = df[['date','title','text','url']]
    df.to_csv('nbc/%03d.csv'%idx, index=False)


 ----------0 ------------

 ----------1 ------------

 ----------2 ------------

 ----------3 ------------

 ----------4 ------------

 ----------5 ------------

 ----------6 ------------

 ----------7 ------------

 ----------8 ------------

 ----------9 ------------

 ----------10 ------------

 ----------11 ------------

 ----------12 ------------

 ----------13 ------------

 ----------14 ------------

 ----------15 ------------

 ----------16 ------------

 ----------17 ------------

 ----------18 ------------

 ----------19 ------------

 ----------20 ------------

 ----------21 ------------

 ----------22 ------------

 ----------23 ------------

 ----------24 ------------

 ----------25 ------------

 ----------26 ------------

 ----------27 ------------

 ----------28 ------------

 ----------29 ------------

 ----------30 ------------

 ----------31 ------------

 ----------32 ------------

 ----------33 ------------

 ----------34 ------------

 ----------35 ------------

 

In [23]:
df_all=[]
for idx in range(50):
    print('\n ----------%s ------------'%idx)
    df = pd.read_csv('nbc/%03d.csv'%idx, header=0)
    df_all.append(df)
df_all = pd.concat(df_all).reset_index(drop=True)
df_all


 ----------0 ------------

 ----------1 ------------

 ----------2 ------------

 ----------3 ------------

 ----------4 ------------

 ----------5 ------------

 ----------6 ------------

 ----------7 ------------

 ----------8 ------------

 ----------9 ------------

 ----------10 ------------

 ----------11 ------------

 ----------12 ------------

 ----------13 ------------

 ----------14 ------------

 ----------15 ------------

 ----------16 ------------

 ----------17 ------------

 ----------18 ------------

 ----------19 ------------

 ----------20 ------------

 ----------21 ------------

 ----------22 ------------

 ----------23 ------------

 ----------24 ------------

 ----------25 ------------

 ----------26 ------------

 ----------27 ------------

 ----------28 ------------

 ----------29 ------------

 ----------30 ------------

 ----------31 ------------

 ----------32 ------------

 ----------33 ------------

 ----------34 ------------

 ----------35 ------------

 

,date,title,text,url
0,2013/03/31,Louisville player after horrific injury: 'Don't worry about me. I'll be OK. You guys go win this thing',"Louisville Cardinals head coach Rick Pitino calls to the referees to stop the game as Cardinals guard Kevin Ware lays on the court with a broken leg in the first half against the Duke Blue Devils during their Midwest Regional NCAA men's basketball game in Indianapolis, Indiana, on March 31, 2013. Jeff Haynes / Reuters\n\nLouisville players talk to guard Kevin Ware after Ware's injury during the first half of the Midwest Regional Final against Duke in the NCAA basketball tournament March 31 in Indianapolis, Ind. Darron Cummings / AP\n\nBy Rob Dauster, CollegeBasketballTalk on NBCSports.com\...",https://www.nbcnews.com/news/photo/louisville-player-after-horrific-injury-dont-worry-about-me-ill-flna1C9155862
1,2013/03/31,Crude oil leaks in Arkansas suburb after ExxonMobil pipeline ruptures,"An ExxonMobil pipeline rupture near Little Rock, Ark., Friday evening has resulted in a “major oil spill,” according to the Environmental Protection Agency -- and ignited further debate over the transportation of crude oil in the U.S.\n\nUp to 10,000 barrels could have been spilled, according to a report filed to the National Response Center by ExxonMobil early Saturday morning.\n\nExxonMobil said Sunday that 12,000 barrels of a mix of oil and water had been recovered, though there was no breakdown on how much of that was oil.\n\n\n\nTwenty-two residents were evacuated from their homes, ac...",https://www.nbcnews.com/business/business-news/crude-oil-leaks-arkansas-suburb-after-exxonmobil-pipeline-ruptures-flna1C9155011
2,2013/03/31,Dust from Chinese storm reaches central California,"Dust from China's Gobi Desert drifted thousands of miles to hang over a central California mountain range this week, according to the Great Basin Unified Air Pollution Control District, a California regional government agency that monitors the environment.\n\nThe massive dust event on March 10 blew sediment from the Gobi Desert across eastern China, prompting health warnings that pollution levels were dangerously high in the country, according to NASA.\n\nThose particles, which have since dissipated, reached Owens Valley, about 225 miles north of Los Angeles and east of the Sierra Nevada m...",https://www.nbcnews.com/news/us-news/dust-chinese-storm-reaches-central-california-flna1C9155498
3,2013/03/31,"Three dead in 95-car pileup near Virginia, North Carolina state line","Three people were killed and more that 20 injured in a massive series of pile-ups involving 95 cars on Interstate 77 near the Virginia and North Carolina state line Sunday afternoon, according to authorities.\n\nThe crashes began around 1 p.m. in the southbound lanes in Carroll County, Virginia, according to Virginia State Police. Excessive fog in the Fancy Gap Mountain area is being blamed for the massive accident.\n\nA state police statement said that there were 17 separate crashes involving 95 vehicles within a one-mile span of the southbound lanes. Three people were killed and about 25...",https://www.nbcnews.com/news/us-news/three-dead-95-car-pileup-near-virginia-north-carolina-state-flna1C9155443
4,2013/03/31,Dominate Your Industry: How to Become the Best in Your Field,"The notion of a miraculous genius being born smarter and more capable than the rest of us mere mortals charms our curiosity. Robert Greene, author of the popular The 48 Laws of Power (Penguin, 2000), would disagree. The fascination we have in prodigies, he says, is ""bogus. It's completely bogus."" Exceptional talent is about hard work, he says.\n\nGreene studied the lives of exceptionally successful people for his latest book, Mastery (Viking/Penguin, 2012). He says that there is no such thing as being born into superior success. Rather, those politicians, entrepreneurs, scientists, athlete...",https://www.nbcnews.com/id/wbna51387883
...,...,...,...,...
344871,2003/01/06,"Gulf War lessons,

In [24]:
df_all.sort_values('date')

,date,title,text,url
344875,2003/01/03,"A $55,000 Net scam warning","He’s a veteran Internet user, and an accomplished dentist. He has a friend in the FBI, and they have discussed Internet crime. Bruce Lachot is not your typical Net scam victim. But in November, just after the birth of his third child, Lachot decided his family needed a larger car. He was tempted by a great deal on a new BMW M5, and optimistically wired money to the German seller. Now, Lachot finds himself out $55,000, with no new sedan, and no chance to recover the money, a victim of one of the most successful and widespread Internet scams to date.\n\nLACHOT AGREED TO SHARE his tale with M...",https://www.nbcnews.com/id/wbna3078503
344872,2003/01/03,Two super cell phones worth waiting for,"In the past few weeks I’ve received half-a-dozen new cellular phones for testing. All of them are nice — but none stood out as something out of the ordinary. So, on a recent trip to London, I thought it was wise to try two phones that aren’t yet available in the United States. I found that, with minor reservations, both are worth waiting for.\n\nInsertArt(1731831)FOR THE PAST TWO YEARS, I’ve been writing about a phone codenamed “Stinger” in various stages of development. Based on Microsoft’s Windows CE/Pocket PC operating system, Stinger was designed from the start to be a combination of P...",https://www.nbcnews.com/id/wbna3078286
344874,2003/01/03,Wall Street’s premature celebration,"Wall Street threw a party to usher in the new year this week, but traders may have let the bubbly go to their heads. Despite a surprisingly strong report indicating an upturn in the manufacturing sector, economists expect another year of slow growth and worry that this week’s exhilarating rally may quickly run its course.\n\nINVESTORS CAN HARDLY be blamed for wanting to celebrate an unexpected jump in the closely watched Purchasing Manager’s Index. The report was one of the first signs of growth after 2002 ended with a run of grim economic news, dismal reports from retailers and steadily d...",https://www.nbcnews.com/id/wbna3073316
344873,2003/01/03,Another hot year for automakers,"America’s top automakers steadily moved vehicles off their lots in December, with sales rising due to zero-percent financing and rebate incentives. Despite the strong month, dealers couldn’t keep pace with 2001, which was the industry’s second-best year on record.\n\nStill the strong December sales are expected to give the U.S. auto industry one of its best years ever. Much of the sales volume was fueled by profit-shrinking incentives, and serious questions linger about prospects for sales in the coming months.\n\nAutomakers around the globe joined in the end-of-year promotions as Detroit’...",https://www.nbcnews.com/id/wbna3073329
344869,2003/01/06,New technique finds farthest known planet,"Researchers say they have spotted a planet thousands of light-years away by watching how it dims the light from the star it orbits. The technique could eventually be used to check millions of stars for the presence of Earthlike planets, a member of the research team said Monday.\n\n“WE BELIEVE the door has been wide open to go and discover a new Earth,” Dimitar Sasselov, an astronomer at the Harvard-Smithsonian Center for Astrophysics, said at this week’s American Astronomical Society meeting in Seattle.\n\nIn the past decade, astronomers have detected 100 planets orbiting stars other than...",https://www.nbcnews.com/id/wbna3077984
...,...,...,...,...
35,2013/03/31,"March 31: Chuck Schumer, Jeff Flake, David Axelrod, Tom Davis, Eugene Robinson, Peggy Noonan, Rob Reiner, Brian Brown, Al Sharpton, Pete Williams","CHUCK TODD:\n\nThis Easter Sunday, the culture war over guns, gay marriage, abortion, and immigration. They're dominating the political debate. After chilling new details emerge from the Newtown shooting that left 20 children dead in December, the president responds to some who claim the push for tighter gun control has 

In [26]:
df_all = df_all[~df_all.text.duplicated()]

In [33]:
# df_others= pd.read_csv('nbc_after2013.csv', header=0)
df_others.date = df_others.date.apply(lambda x: re.sub('(\d\d\d\d)(\d\d)(\d\d)', '\\1/\\2/\\3', str(x)))
df_others

,url,date,title,authors,text
0,https://www.nbcnews.com/politics/politics-news/north-korea-fired-least-one-missile-over-weekend-u-s-n1261860,2021/03/24,"North Korea fired at least one missile over the weekend, U.S. officials say","['Andrea Mitchell', 'Andrea Mitchell Is Chief Washington Correspondent', 'Chief Foreign Affairs Correspondent For Nbc News.', 'Dan De Luce', 'Dan De Luce Is A Reporter For The Nbc News Investigative Unit.', 'Abigail Williams', 'Abigail Williams Is A Producer', 'Reporter For Nbc News Covering The State Department.', 'Dareh Gregorian', 'Dareh Gregorian Is A Politics Reporter For Nbc News.']","WASHINGTON — North Korea fired at least one missile over the weekend, two U.S. officials told NBC News Tuesday.\n\nIt marks the first report of such activity since Joe Biden was sworn in as president.\n\nOfficials declined to say what type of missile was fired or where it landed. It was unclear why South Korea’s government had yet to comment on the missile launch. Officials in Seoul typically issue statements after North Korean missile or nuclear tests, and the North Korean government has been known to brag about them, as well.\n\nAsked by reporters what he could say about the incident, Bi..."
1,https://www.nbcnews.com/news/us-news/ex-nypd-officer-allegedly-shaking-tambourine-during-capitol-riot-charged-n1261829,2021/03/24,Ex-NYPD officer said to have shaken tambourine during Capitol riot is charged,"['Tim Fitzsimons', 'Tim Fitzsimons Is A Reporter For Nbc News.']","A retired New York police officer who is alleged to have played a tambourine in the halls of the U.S. Capitol during the Jan. 6 riot has been charged with several misdemeanors, the FBI said.\n\nThe former officer, Sara Carpenter, 51, turned herself in to federal authorities and appeared before a federal judge in Brooklyn on Tuesday, NBC New York reported.\n\nThe FBI said Sara Carpenter, a former New York police officer from Queens, was inside the Capitol during the siege on Jan. 6. FBI\n\nCarpenter is charged with knowingly entering or remaining in any restricted building or grounds withou..."
2,https://www.nbcnews.com/politics/justice-department/justice-department-investigating-prosecutor-60-minutes-interview-about-capitol-riot-n1261883,2021/03/24,Justice Department investigating prosecutor for '60 Minutes' interview about Capitol riot cases,"['Pete Williams', 'Pete Williams Is An Nbc News Correspondent Who Covers The Justice Department', 'The Supreme Court', 'Based In Washington.']","A federal prosecutor told a judge Tuesday that the Justice Department is investigating Michael Sherwin, the former acting U.S. attorney for Washington, D.C., for comments he made during a ""60 Minutes"" interview about the Capitol riot investigations.\n\nIn an interview broadcast Sunday on CBS, Sherwin said he believed the evidence was ""trending toward"" meeting the standard to file sedition charges against some of the rioters. ""I believe the facts do support those charges,"" he said, and he also made other comments about individual defendants.\n\nHis statements struck some former prosecutors ..."
3,https://www.nbcnews.com/news/us-news/nurse-correctional-officer-killed-assault-inmate-iowa-penitentiary-n1261882,2021/03/24,Nurse and correctional officer killed in assault by inmate at Iowa penitentiary,['John Cheang'],"A correctional officer and a nurse were killed after an inmate attacked multiple staff members and other inmates at the Anamosa State Penitentiary in Iowa, prison authorities said.\n\nThe attack took place Tuesday morning in the prison’s infirmary. The inmate was restrained by additional security staff, while other employees of the facility began “attempting life-saving first aid on the injured until paramedics arrived,” the Iowa Department of Corrections said in a statement.\n\nA correctional nurse and correctional officer died of their injuries, the statement said.\n\nThe names of the de..."
4,https://www.nbcnews.com/news/latino/latina-author-isabel-allend

In [37]:
df_merge = pd.concat([df_all, df_others]).sort_values('date')

In [40]:
df_merge = df_merge[~df_merge.text.duplicated()]
df_merge = df_merge.sort_values('date')
df_merge = df_merge.iloc[:,:4]
df_merge

,date,title,text,url
344875,2003/01/03,"A $55,000 Net scam warning","He’s a veteran Internet user, and an accomplished dentist. He has a friend in the FBI, and they have discussed Internet crime. Bruce Lachot is not your typical Net scam victim. But in November, just after the birth of his third child, Lachot decided his family needed a larger car. He was tempted by a great deal on a new BMW M5, and optimistically wired money to the German seller. Now, Lachot finds himself out $55,000, with no new sedan, and no chance to recover the money, a victim of one of the most successful and widespread Internet scams to date.\n\nLACHOT AGREED TO SHARE his tale with M...",https://www.nbcnews.com/id/wbna3078503
344874,2003/01/03,Wall Street’s premature celebration,"Wall Street threw a party to usher in the new year this week, but traders may have let the bubbly go to their heads. Despite a surprisingly strong report indicating an upturn in the manufacturing sector, economists expect another year of slow growth and worry that this week’s exhilarating rally may quickly run its course.\n\nINVESTORS CAN HARDLY be blamed for wanting to celebrate an unexpected jump in the closely watched Purchasing Manager’s Index. The report was one of the first signs of growth after 2002 ended with a run of grim economic news, dismal reports from retailers and steadily d...",https://www.nbcnews.com/id/wbna3073316
344873,2003/01/03,Another hot year for automakers,"America’s top automakers steadily moved vehicles off their lots in December, with sales rising due to zero-percent financing and rebate incentives. Despite the strong month, dealers couldn’t keep pace with 2001, which was the industry’s second-best year on record.\n\nStill the strong December sales are expected to give the U.S. auto industry one of its best years ever. Much of the sales volume was fueled by profit-shrinking incentives, and serious questions linger about prospects for sales in the coming months.\n\nAutomakers around the globe joined in the end-of-year promotions as Detroit’...",https://www.nbcnews.com/id/wbna3073329
344872,2003/01/03,Two super cell phones worth waiting for,"In the past few weeks I’ve received half-a-dozen new cellular phones for testing. All of them are nice — but none stood out as something out of the ordinary. So, on a recent trip to London, I thought it was wise to try two phones that aren’t yet available in the United States. I found that, with minor reservations, both are worth waiting for.\n\nInsertArt(1731831)FOR THE PAST TWO YEARS, I’ve been writing about a phone codenamed “Stinger” in various stages of development. Based on Microsoft’s Windows CE/Pocket PC operating system, Stinger was designed from the start to be a combination of P...",https://www.nbcnews.com/id/wbna3078286
344871,2003/01/06,"Gulf War lessons, learned or not","The Gulf War is remembered as a walkover by many Americans — a late 20th-century blitzkrieg. In reality, the U.S. military had a legion of troubles in the 1990-91 conflict, many of which caused unnecessary deaths to allied troops and Iraqi civilians alike. Other tragedies were averted merely by luck or Iraqi incompetence. What new tactics, technologies and procedures are in place today to ensure that some of the worst mistakes of the first Gulf War will not be repeated in a second Iraq war?\n\nThe Gulf War of popular memory bears little resemblance to the one fought by the United States an...",https://www.nbcnews.com/id/wbna3071782
...,...,...,...,...
8,2021/03/24,Why the casting of an Indian British lead in 'Bridgerton' is historically accurate,"When the British actress Simone Ashley was cast as the romantic lead in the upcoming second season of Netflix’s breakout hit drama “Bridgerton,” many fans of South Asian descent were thrilled — because it means that viewers of historical costume dramas will finally get the chance to see a South Asian woman in a leading romantic role.\n\n“I feel like I’ve never seen a brown main character in a

In [ ]:
df_merge.to_csv('nbc_2001_2021.csv', index=False)

In [64]:
df

,date,title,text,url
0,1998-01-31,Lewinsky's Lawyer: Tripp Wasn't Privy To Any Clinton Call,"Lewinsky's Lawyer: Tripp Wasn't Privy To Any Clinton Call Former intern to visit dad in California; no immunity deal in sight Tripp WASHINGTON (AllPolitics, Jan. 31) -- Monica Lewinsky's attorney has contradicted a claim by Linda Tripp that she overheard part of a late-night telephone conversation between the former White House intern and President Bill Clinton. In an interview aired Friday night on ABC's ""20/20,"" lawyer William Ginsburg said, ""Based on my investigation of the entire situation, Ms. Tripp was never privy to any conversation that Monica Lewinsky ever had with the president o...",https://www.cnn.com/ALLPOLITICS/1998/01/31/clinton.lewinsky2/
1,1998-01-31,Keeping Score In the Lewinsky Matter,"Keeping Score In the Lewinsky Matter What do we know, and how do we know it, in the Monica Lewinsky matter? Updated: 01-31-98 ""Gossip, innuendo and hearsay are being passed off as fact. Allegations by unnamed sources are claimed to be credible."" -- Bob Bennett, Bill Clinton's personal attorney The story of the alleged affair between President Bill Clinton and former White House intern Monica Lewinsky exploded with a torrent of information -- facts, unproven statements and outrageous rumors. Here's a guide to help you discern fact from fiction, as well as what falls somewhere in between, an...",https://www.cnn.com/ALLPOLITICS/1998/01/31/credibility/
2,1998-01-31,Clinton Plays Golf Near Presidential Retreat,"Clinton Plays Golf Near Presidential Retreat\n\nReporters kept far away; no questions about Lewinsky\n\nTHURMONT, Md. (AllPolitics, Jan. 31) -- President Bill Clinton emerged from seclusion at Camp David Saturday for a round of golf with friends, relatives and his dog Buddy.\n\nJoining the president was his brother Roger Clinton, brothers-in-law Hugh and Tony Rodham and Hollywood producer Harry Thomasson. The first dog amused himself by romping on the greens and fairways, and introducing himself to strangers nearby. Buddy seemed to take particular pleasure in moving golf balls from their p...",https://www.cnn.com/ALLPOLITICS/1998/01/31/golf/
3,1998-01-31,Forbes Wins A Straw Poll,"Forbes Wins A Straw Poll\n\nARLINGTON, Va. (AllPolitics, Jan. 31) -- Millionaire publisher Steve Forbes emerged as the preferred presidential candidate in a straw poll of conservatives surveyed at the annual Conservative Political Action Conference here.\n\nForbes had the support of 23 percent of the 1,500 conferees, followed by Texas Gov. George W. Bush with 10 percent and commentator Pat Buchanan and House Speaker Newt Gingrich with 7 percent each. Another dozen Republicans trailed the top four. Thirteen percent of people who were surveyed said they were undecided.\n\nThe survey was take...",https://www.cnn.com/ALLPOLITICS/1998/01/31/forbes/
4,1998-01-31,Ken Starr's Law Firm Subpoenaed By Clinton Lawyers,"Ken Starr's Law Firm Subpoenaed By Clinton Lawyers By John King/CNN WASHINGTON (Jan. 31) -- President Bill Clinton's private attorney has subpoenaed the law firm of Whitewater Independent Counsel Ken Starr to determine if it has done any work in the Paula Jones civil rights-sexual harassment case, sources tell CNN. Starr still works at Kirkland & Ellis for a reported $1 million a year, and if the firm was involved in the Jones cause it would raise questions about Starr's objectivity as independent counsel. Sources say Clinton's attorney, Bob Bennett, issued his subpoena after discovering t...",https://www.cnn.com/ALLPOLITICS/1998/01/31/subpoena.king/
...,...,...,...,...
27694,2012-11-26,Connecticut task force crafts gun law overhaul,"Supporters of gun control hold a candlelight vigil for victims of the shooting outside the White House.\n\nWashington resident Rachel Perrone, left, and her 5-year-old son, Joe, center, join others outside the White House in a candlelight vigil.\n\nObama wipes tears as he makes a statement in response to the shooting on 